# 유전 알고리즘 

**변수 선택**
- 변수 선택이란 데이터가 갖고 있는 많은 변수, Features 중에서 최선의 변수 조합을 찾아내는 기법 
- 차원 축소 방법론 중 하나이며 주로 Supervised 방법론이 많음 
-  종류로는 전진 선택법, 후진 제거법, 단계선택법 그리고 유전 알고리즘 등이 있다. 

**유전 알고리즘**

Overview 
- 전진 선택법, 후진 제거법, 단계 선택법과 같은 지도 차원 축소 방법은 변수의 모든 부분 집합을 대상으로 최적해를 탐색하는 것이 아닌 일부의 부분집합만을 대상으로 탐색을 진행하는 local search에 해당한다. 
- Local search는 연산속도 측면에서 효율적인 장점이 있지만 한편으로는 local minimum에 빠질 수 있다라는 문제가 존재한다. 
- 따라서 기존의 local search에 한계점을 보완함으로써 성능을 향상시키고자 하는 것이 유전 알고리즘의 목적이다. 

용어 
- 염색체 : 유전 알고리즘에서는 변수 선택을 나타내기 위한 표현, 변수 조합 
- 유전자 : 염색체에 존재하는 각 변수의 변수 선택 여부 인코딩 정보(선택=1,미선택=0)
- 자손 : 부모 염색체 일부를 섞는 crossover 과정을 통해 생성된 염색체 
- 적합도 : 각 염색체로 모델링 후의 성능 

순서 

1. 염색체 초기화 및 파라미터 설정 
2. 각 염색체 선택, 변수 별 모델 학습 
3. 염색체 적합도 평가 (성능 평가)
4. 우수 염색체 선택 
5. 다음 세대 염색체 생성 (crossover,mutation)
6. 최종 변수 집합 선택 


Sklearn 의 wine 데이터셋을 기반으로 진행 

In [103]:
import pandas as pd 
import numpy as np 
from sklearn.datasets import load_wine,load_diabetes
from tqdm import tqdm 
import random 

In [104]:
data = load_diabetes()
#target_name = data.target_names
features = np.array(data.feature_names)
num_features = len(features)
#print(target_name)
print(num_features)

#데이터 로드  
df = pd.DataFrame(data.data)
df.columns = data['feature_names']
df['class'] = data['target']
df = df.sample(frac=1,random_state=42).reset_index(drop=True)

10


# Step 1 : 염색체 초기화 및 파라미터 설정 
- Data의 Features 수에 맞게 Chromosome list를 생성하여 초기 Population을 만듦
- 각 Chromosome은 Random으로 생성된 Binary Integer로 채워 짐 

In [150]:
#Wine 데이터셋의 Features 는 13개 
num_population = 50 # 초기 population 인구 지정 

population = [np.random.randint(0,2,num_features) for _ in range(num_population)]
population

[array([1, 0, 0, 1, 1, 0, 1, 0, 1, 0]),
 array([0, 0, 1, 1, 0, 1, 0, 0, 0, 1]),
 array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0]),
 array([0, 1, 0, 0, 0, 1, 0, 1, 0, 1]),
 array([1, 1, 1, 0, 1, 1, 0, 0, 0, 0]),
 array([0, 1, 0, 0, 1, 0, 0, 0, 1, 0]),
 array([1, 0, 0, 0, 1, 1, 0, 0, 0, 1]),
 array([1, 1, 0, 0, 0, 1, 1, 1, 1, 0]),
 array([0, 1, 1, 1, 0, 1, 1, 0, 0, 0]),
 array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 1, 0, 0, 0, 1, 1, 0, 0, 1]),
 array([0, 0, 1, 0, 1, 0, 1, 1, 0, 0]),
 array([1, 1, 0, 0, 0, 1, 0, 1, 0, 1]),
 array([1, 0, 1, 1, 1, 0, 0, 0, 0, 1]),
 array([1, 0, 0, 0, 1, 1, 0, 1, 0, 1]),
 array([1, 0, 0, 1, 1, 1, 1, 1, 1, 0]),
 array([0, 0, 1, 0, 0, 1, 0, 0, 1, 1]),
 array([0, 1, 1, 0, 1, 0, 0, 0, 1, 1]),
 array([1, 0, 1, 0, 1, 1, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 1, 0, 1, 1, 1, 0]),
 array([0, 1, 0, 0, 1, 1, 0, 1, 0, 1]),
 array([0, 1, 1, 1, 0, 1, 0, 0, 1, 0]),
 array([0, 0, 0, 1, 1, 1, 0, 0, 1, 0]),
 array([0, 1, 1, 0, 0, 1, 0, 1, 1, 1]),
 array([0, 0, 0, 0, 1, 1, 1, 1, 1, 0]),


# Step 2 : Model Fitting 
- 1에서 생성된 population의 각 Chromosome을 이용하여 모델을 학습 및 평가 진행 

In [123]:
#population에서 유전자 선택 
chromosome = population[0]

#유전자로 Feature 선택 
def feature_selection(chromosome):
    return features[np.where(chromosome)[0]]
    
#선택된 Feature로 x 데이터 생성 
selected_features = feature_selection(chromosome)

def selected_dataset(selected_features):
    X = df[selected_features].values
    Y = df['class'].values.reshape(-1,1)

    #Train - Test 데이터셋 분리 
  
    train_x,test_x,train_y,test_y = train_test_split(X,Y,test_size=0.2,random_state=42,shuffle=True)
    return [train_x,test_x,train_y,test_y]

#평가할 모델 생성, 학습, 평가 진행 
#평가는 R^2를 사용 
from sklearn.linear_model import LinearRegression
def model_scoring(datasets):
    [train_x,test_x,train_y,test_y] = datasets
    model = LinearRegression()
    model.fit(train_x,train_y)
    score = model.score(test_x,test_y)
    return score 

위 과정을 모든 Chromosome에 대하여 진행 

In [151]:
score_list = [] 
for chromosome in population:
    selected_features = feature_selection(chromosome)
    datasets = selected_dataset(selected_features)
    score = model_scoring(datasets)
    score_list.append(score)

# Step 4: 우수 염색체 선택 
- Score가 우수한 염색체를 선택하여 부모 세대로 명명 
- 선택 된 부모 세대를 통해 교배 및 돌연변이 생성 가능 
- 선택 된 방법은 상위 x%를 선택하거나 확률적으로 Score의 %에 따라 랜덤하게 선택 가능 

In [152]:
selected_ratio = 0.4

ratio_selected_for_parent = int(np.round(num_population)*selected_ratio)

parent_chromosome_index = pd.Series(score_list).sort_values(ascending=False).index[:ratio_selected_for_parent]

parent_chromosome = np.array(population)[parent_chromosome_index]

# Step 5: Crossover & Mutation 
- 유전자 교배 및 돌연변이를 생성하여 새로운 population 생성 

Crossover 
- Candidate Population에서 random 하게 2개의 부모 선택 
- 선택된 부모 Chromosome들을 2개 선택 해 각각의 유전자 정보를 교환한다. 

In [241]:
def crossover_index(selected_parents):
    prob = np.random.random_sample(num_features)
    parent_1_index = np.round(prob)
    parent_2_index = np.abs(parent_1_index-1)
    child_1_index = selected_parents.iloc[0,:].values*parent_1_index + selected_parents.iloc[1,:].values*parent_2_index
    child_2_index = selected_parents.iloc[1,:].values*parent_1_index + selected_parents.iloc[0,:].values*parent_2_index
    return child_1_index, child_2_index 

num_parents =2 
selected_parents = pd.DataFrame(parent_chromosome).sample(num_parents)
child_1,child_2 = crossover_index(selected_parents)
print(selected_parents)

    0  1  2  3  4  5  6  7  8  9
6   0  1  0  1  1  1  1  0  0  1
13  1  0  1  0  0  0  0  0  1  0


In [252]:
print(pd.DataFrame([selected_parents.iloc[0,:],selected_parents.iloc[1,:],prob]))

                  0         1         2         3         4         5  \
6          0.000000  1.000000  0.000000  1.000000  1.000000  1.000000   
13         1.000000  0.000000  1.000000  0.000000  0.000000  0.000000   
Unnamed 0  0.362088  0.380878  0.695685  0.157083  0.022695  0.199793   

                 6         7         8        9  
6          1.00000  0.000000  0.000000  1.00000  
13         0.00000  0.000000  1.000000  0.00000  
Unnamed 0  0.32432  0.296837  0.270262  0.96095  


In [253]:
print(pd.DataFrame([child_1,child_2]))

     0    1    2    3    4    5    6    7    8    9
0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0
1  0.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0


Mutation 
- 2개의 Child Chromosome에 대하여 random하게 값을 Flip 
- Crossover 처럼 random 값을 부여한 뒤 
- hyperparmeter Threshold값에 의해 Mutation 진행 

In [294]:


def mutant(child)
    prob = np.random.random_sample(num_features)
    mutant = pd.Series(prob).apply(lambda x : Thresholding(x) ).values        
    child = np.abs(child +mutant-1)
    return child 

child_1 = mutant(child_1)
child_2 = mutant(child_2)

In [300]:
score_list = [] 
for chromosome in [child_1,child_2]:
    selected_features = feature_selection(chromosome)
    datasets = selected_dataset(selected_features)
    score = model_scoring(datasets)
    score_list.append(score)
score_list    

[0.2819370237331775, 0.3205616715875692]

# step 6: Final step : Generation 반복 
- 최적의 변수를 찾기 위해서 여러 Generation에 거쳐 2~5반복 작업 진행 

In [319]:
from sklearn.model_selection import train_test_split
class Genetic_algorithm:
    def __init__(self,df,num_population,selected_ratio,mutant_prob):
        self.df = df 
        self.features = df.drop(columns='class').columns
        self.num_features = len(self.features)
        #Hyper parameter 
        self.num_population = num_population
        self.selected_ratio = selected_ratio
        self.mutant_prob = mutant_prob
        

    def feature_selection(self,chromosome):
        return self.features[np.where(chromosome)[0]]

    def selected_dataset(self,selected_features):
        X = self.df[selected_features].values
        Y = self.df['class'].values.reshape(-1,1)

        #Train - Test 데이터셋 분리 
        train_x,test_x,train_y,test_y = train_test_split(X,Y,test_size=0.2,random_state=42,shuffle=True)
        return [train_x,test_x,train_y,test_y]

    def model_scroing(self,datasets):
        [train_x,test_x,train_y,test_y] = datasets
        self.train_x = train_x 
        self.train_y = train_y 
        model = LinearRegression()
        model.fit(train_x,train_y)
        score = model.score(test_x,test_y)
        return score 

    def step_2_population_eval(self,population):
        score_list = [] 
        for chromosome in population:
            selected_features = self.feature_selection(chromosome)
            datasets = self.selected_dataset(selected_features)
            score = model_scoring(datasets)
            score_list.append(score)
        return np.array(score_list)

    def step_4(self,score_list):
        ratio_selected_for_parent = int(np.round(self.num_population)*self.selected_ratio)

        parent_chromosome_index = pd.Series(score_list).sort_values(ascending=False).index[:ratio_selected_for_parent]

        parent_chromosome = np.array(self.population)[parent_chromosome_index]
        return parent_chromosome

    def step_5_crossover(self,selected_parents):
        prob = np.random.random_sample(self.num_features)
        parent_1_index = np.round(prob)
        parent_2_index = np.abs(parent_1_index-1)
        child_1_index = selected_parents.iloc[0,:].values*parent_1_index + selected_parents.iloc[1,:].values*parent_2_index
        child_2_index = selected_parents.iloc[1,:].values*parent_1_index + selected_parents.iloc[0,:].values*parent_2_index
        return child_1_index, child_2_index 
    
    def step_5_mutant(self,child):

        def Thresholding(x):
            Threshold = self.mutant_prob
            if x >= Threshold:
                x = 1 
            if x < Threshold:
                x = 0 
            return x 

        prob = np.random.random_sample(self.num_features)
        mutant = pd.Series(prob).apply(lambda x : Thresholding(x) ).values        
        child = np.abs(child +mutant-1)
        return child 

    def __call__(self,Generation):
        best_generation = []
        for g in range(Generation):
        #step 1 : 초기 Population init 
            self.population = [np.random.randint(0,2,self.num_features) for _ in range(self.num_population)]
        #step 2,3 : 초기 Population들 각각 평가 
            score_list = self.step_2_population_eval(self.population)
        #step 4  : 상위 우수 인자 분별 
            parent_chromosome = self.step_4(score_list)
        #step 5 : 부모 선택 및 교배, 돌연변이 
            #부모 선택 
            num_parents =2 
            selected_parents = pd.DataFrame(parent_chromosome).sample(num_parents)
            #corssover 
            child_1, child_2 = self.step_5_crossover(selected_parents)
            #Mutant         
            child_1 = self.step_5_mutant(child_1)
            child_2 = self.step_5_mutant(child_2)

            score_list = [] 
            for chromosome in [child_1,child_2]:
                try:
                    selected_features = self.feature_selection(chromosome)
                    datasets = self.selected_dataset(selected_features)
                    score = model_scoring(datasets)
                    score_list.append(score)
                except:
                    pass 
            
            best_child = np.argmax(score_list)
            best_score = score_list[best_child]
            if g == 0:
                Survived_child = [child_1,child_2][best_child]
                Survived_score = best_score 
            else: 
                if Survived_score < best_score:
                    Survived_child = [child_1,child_2][best_child]
                    Survived_score = best_score 
        return Survived_child,Survived_score

In [330]:
num_population = 50 
selected_ratio = 0.4 
num_generation = 20
mutant_prob = 0.2
ga =  Genetic_algorithm(df,num_population,selected_ratio,mutant_prob)
best_child, best_score = ga(num_generation)

print(f'best score = {best_score}')
print(f'best Features = {feature_selection(best_child)}')

best score = 0.3777644684752962
best Features = ['sex' 'bp' 's1' 's3' 's4' 's6']
